# Imports

In [38]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn import metrics
# import cvxopt # <- installation via conda recommended
from collections import defaultdict
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import nltk
import scipy.optimize as sopt
import scipy.stats as sstats
import csv

# The goal

### TODO

# Data loading

We will be using LOB (Limit Order Book) data from London stock market, September 2013.
Every row of our data represent all active ask and bid orders in some moment on time. Row can be describe as below:

date time 'BID' $p_{b1}$ $w_{b1}$ $p_{b2}$ $w_{b2}$ ... $p_{bn}$ $w_{bn}$ 'ASK' $p_{a1}$ $w_{a1}$ $p_{a2}$ $w_{a2}$ ... $p_{am}$ $w_{am}$,
where $p_b$, $w_b$ are prices and size of bid order and $p_a$, $w_a$ are prives and sizes of ask order. Prices $p_x$ are sorted ascending.

LOB data are often represented as 3-element tuples $(p_x,w_x,t_x)$, where $p_x,w_x,t_x$ represent price,size and time of $x-th$ order and $w_x$ is greater than zero for ask order.

In our case it will be batter to represent data as a list in which every element is tuple of bid and ask orders lists. Bid and ask lists consist of $(p_x,w_x)$ tuples, and $w_x > 0$ for all orders.

We consider orders from $8:30$ to $16:30$ to eliminate abnormal trading behaviour that can occur shortly after the opening auction or shortly before closing auction.





In [2]:
def load_data(path,start_time=83000000,stop_time=163000000):
    X = []
    with open(path,newline='') as file:
        csv_reader = csv.reader(file,delimiter='\t')
        for row in csv_reader:
            date,time = map(int,row[0].split(' '))
            if time < start_time or time > stop_time:
                continue
            
            line = 2
            ASK_list = []
            BID_list = []
            while line < len(row):
                if row[line] == 'ASK':
                    break
                p,w = map(float,row[line:line+2])
                BID_list.append((p,w))
                line += 2
            line += 1
            while line < len(row):
                p,w = map(float,row[line:line+2])
                ASK_list.append((p,w))
                line += 2
            
            X.append((BID_list,ASK_list))

    return X 

In [3]:
path = "C:\Projekt_ED\OrderBookSnapshots.csv"
data = load_data(path)

In [4]:
len(data)

4810

# Data process functions

At a given time $t$, the bid price $b(t)$ is the highest stated price among active buy orders,  
<center>$b(t) = \max_{x \in BIDlist(t)} p_x $</center>  
and the ask price $a(t)$ is the lowest stated price among active sell orders,  
<center>$a(t) = \min_{x \in ASKlist(t)} p_x $</center>  
The mid price at time $t$ is  
<center>$m(t) = \frac{a(t)+b(t)}{2} $</center>  
  
The bid size $n_b(t)$ is total size of active buy orders with price equal to bid price  
<center>$n_b(t) = \sum_{x \in BIDlist(t) | px = b(t)} w_x $</center>  
and ask size $n_b(t)$ is total size of active sell orders with price equal to ask price  
<center>$n_a(t) = \sum_{x \in ASKlist(t) | px = a(t)} w_x $</center>  
  
At a given time $t$, the queue imbalance $I(t)$ is normalized difference between $n_b(t)$ and $n_a(t)$  
<center>$I(t) = \frac{n_b(t) - n_a(t)}{n_b(t) + n_a(t)} $</center>  
  


We can expend those definitions considering k highest (lowest) bid (ask) prices.
<center>$b_k(t) = k-th$ highest price $\in BIDlist(t)$</center>  
   
<center>$a_k(t) = k-th$ lowest price $\in ASKlist(t)$</center>  
  
<center>$n_{k,b}(t) = \sum_{x \in BIDlist(t) | px 	\geqslant b_k(t)} w_x $</center>  
  
<center>$n_{k,a}(t) = \sum_{x \in ASKlist(t) | px \leqslant a_k(t)} w_x $</center>  
  
At a given time $t$, the $k-th$ queue imbalance $I_k(t)$ is normalized difference between $n_{k,b}(t)$ and $n_{k,b}(t)$  
<center>$I_k(t) = \frac{n_{k,b}(t) - n_{k,a}(t)}{n_{k,b}(t) + n_{k,a}(t)} $</center>  


In [5]:
def bid_price(data,t):
    return data[t][0][-1][0]

In [6]:
def ask_price(data,t):
    return data[t][1][0][0]

In [7]:
def mid_price(data,t):
    return (bid_price(data,t) + ask_price(data,t))/2

In [8]:
def bid_size(data,t):
    return data[t][0][-1][1]

In [9]:
def ask_size(data,t):
    return data[t][1][0][1]

In [10]:
def queue_imbalance(data,t):
    nb = bid_size(data,t)
    na = ask_size(data,t)
    return (nb-na)/(nb+na)

In [11]:
def queue_imbalance_k(data,t,k=2):
    sb = 0
    sa = 0
    for i in range(k):
        sb += data[t][0][-(i+1)][1]
        sa += data[t][1][i][1]
    return (sb-sa)/(sb+sa)

# Target defining

Dejwo opisz to dobrze
<center>$T = [t_x | m(t_x) \neq m(t_{x-1})] $</center>
<center>$N = |T| $</center>
<center>where $t_x$ are next times in our snapshots data</center>
  
<center>$targets = [1$ if $m(t_{x+1}) > m(t_{x})$ else $0$ for $t_x \in T]$</center>

In [12]:
def get_time_and_target(data):
    T = [0]
    target = []
    for t in range(1,len(data)):
        t_1 = T[-1]
        mt = mid_price(data,t)
        mt_1 = mid_price(data,t_1)
        if mt != mt_1:
            T.append(t)
            if mt > mt_1:
                target.append(1)
            else:
                target.append(0)
    return np.array(T[:-1]),np.array(target)

In [13]:
T,target = get_time_and_target(data)

# Data matrix defining

Now we can define our data matrix.
\begin{bmatrix}
    I_1(t_0) & I_2(t_0) &I_3(t_0) & \dots  & I_K(t_0) \\
    I_1(t_1) & I_2(t_1) &I_3(t_1) & \dots  & I_K(t_1) \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    I_1(t_N) & I_2(t_N) &I_3(t_N) & \dots  & I_K(t_N) \\
\end{bmatrix}
  
We can notice, that for $K=1$ our data matrix is equal to:
\begin{bmatrix}
    I(t_0) \\
    I(t_1) \\
    \vdots \\
    I(t_N) \\
\end{bmatrix}

In [15]:
K = 2
X = np.array([[queue_imbalance_k(data,t,k) for k in range(1,K+1)] for t in T])

## Train test split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(
X, target, test_size=0.2, random_state=42,shuffle=False)

In [32]:
def print_score(preds,Y,name):
    print(name)
    acc = np.mean(preds == Y)
    print(f"Acc: {acc}")
    M = metrics.confusion_matrix(preds,Y)
    N = np.sum(M)
    print('\nConfusion matrix:')
    print(M)
    print(f'\nTrue negative (rating = 1): {M[0][0]}')
    print(f'True positive (rating = 10): {M[1][1]}')
    print(f'False negative: {M[0][1]}')
    print(f'False positive: {M[1][0]}')
    return M,N,acc

# Logistic regression

Our goal is to predict if $m_{t+1} > m_t$ using data vector
\begin{bmatrix}
    I_1(t) & I_2(t) &I_3(t) & \dots  & I_K(t) \\
\end{bmatrix}

We will be using logistic regression.
Using logistic regression we can calculate probability of the sample $x$ belonging to class 1
<center>$p(y=1|x) = \sigma(\theta^Tx) = \frac{1}{1 + e^{-\theta^Tx}}$</center>  
  
We can observe that:
$ \begin{align} p(y=y^{(i)}|x^{(i)};\Theta) &= \cases{p(y=1|x;\Theta) &if $y^{(i)}=1$ \\ 1-p(y=1|x;\Theta) &if $y^{(i)}=0$} \\ &= p(y=1|x;\Theta)^{y^{(i)}}(1-p(y=1|x;\Theta))^{(1-y^{(i)})} \\ &= \sigma(\Theta^Tx)^{y^{(i)}}(1-\sigma(\Theta^Tx))^{(1-y^{(i)})} \end{align} $
  
Therefore the negative log likelihood ($nll$) is:$$
\begin{split}
nll(\Theta) &= -\sum_{i=1}^{N} y^{(i)} \log \sigma(\Theta^Tx) + (1-y^{(i)})\log(1-\sigma(\Theta^Tx)) = \\
&= -\sum_{i=1}^{N}y^{(i)}\log p(y=1|x^{(i)}; \Theta) + (1-y^{(i)})\log  p(y=0|x^{(i)}; \Theta)
\end{split}
$$

So we are searching for $\theta$:
<center>$\theta = arg\,max_{\theta}$ $nll(\theta) $</center>  
  
We can consider also logistic regression with regularization, where:$$
\begin{split}
nll(\Theta) &= -\sum_{i=1}^{N}y^{(i)}\log p(y=1|x^{(i)}; \Theta) + (1-y^{(i)})\log  p(y=0|x^{(i)}; \Theta) + \frac{\lambda}{2} \sum_{i}\theta_{i}^{2}
\end{split}
$$

There are few ways to find $\theta$. We will consider Newtod-Raphson Method and L-BFGS-B solver. We will compare this with sklearn LogisticRegression.

## L-BFGS-B solver

In [25]:
class Logistic_Regression:
    def __init__(self, max_iter=500, solver_calls=5,lambda_ = 0.5,Theta=None, solver=sopt.fmin_l_bfgs_b, debug=False):
        self.Theta = Theta
        self.solver_calls = solver_calls
        self.max_iter = max_iter
        self.solver = solver
        self.debug = debug
        self.lambda_ = lambda_
    
    def __sigmoid(self,x):
        return 1 / (1 + np.exp(-x))    
    
    def __logreg_loss(self, Theta, X, Y):
        Theta = Theta.astype(np.float64)
        X = X.astype(np.float64)
        Y = Y.astype(np.float64)
        
        if self.debug:
            print(f"Loss calculating... ",end="")
        Z = np.dot(Theta,X.T)
        if self.debug:
            print(f" Z done... ",end="")
        SZ = self.__sigmoid(Z)
        Y_ = Y[:,np.newaxis]
        nll = -np.sum((Y_*np.log2(SZ+1e-50) + (1-Y_)*np.log2(1-SZ+1e-50)))
        nll += (self.lambda_/2) * np.sum(Theta**2)
        if self.debug:
            print(f" nll done... ",end="")
        grad = np.dot(X.T, (SZ - Y).T ) / len(Y)
        grad = grad.reshape(Theta.shape) + self.lambda_ * Theta
        if self.debug:
            print(f" grad done... done ")
        return nll, grad
    
    def fit(self,X,y):
        Theta = self.Theta
        if Theta is None:
            Theta = np.ones(X.shape[1]+1)
        
        X_with_ones = np.hstack((np.ones((X.shape[0],1)),X))
      
        for i in tqdm(range(self.solver_calls), desc='Calculating Theta', position=0):
            Theta = self.solver(lambda th: self.__logreg_loss(th, X_with_ones, y), 
                                Theta, maxiter=self.max_iter)[0]
        self.Theta = Theta

    def predict(self,X):
        X_with_ones = np.hstack((np.ones((X.shape[0],1)),X))
        preds = (np.dot(self.Theta,X_with_ones.T) >= 0) * 1
        return preds

In [33]:
LR_solver = Logistic_Regression()

In [42]:
LR_solver.fit(X_train,y_train)
preds_train_solver = LR_solver.predict(X_train)

Calculating Theta: 100%|█████████████████████████████████████████████████████████████████| 5/5 [00:17<00:00,  3.57s/it]


In [40]:
M,N,acc = print_score(preds_train_solver,y_train,
                      'Train data, L-BFGS-B solver, lambda=0.5')

Train data, L-BFGS-B solver, lambda=0.5
Acc: 0.5395040369088812

Confusion matrix:
[[ 739  628]
 [ 969 1132]]

True negative (rating = 1): 739
True positive (rating = 10): 1132
False negative: 628
False positive: 969


In [43]:
preds_test_solver = LR_solver.predict(X_test)

In [44]:
M,N,acc = print_score(preds_test_solver,y_test,
                      'Test data, L-BFGS-B solver, lambda=0.5')

Test data, L-BFGS-B solver, lambda=0.5
Acc: 0.5518433179723502

Confusion matrix:
[[172 141]
 [248 307]]

True negative (rating = 1): 172
True positive (rating = 10): 307
False negative: 141
False positive: 248


## Newtod-Raphson Method

### TODO

Newton's method is an iterative method for finding the roots of a differentiable function $F$, which are solutions to the equation $F(x) = 0$. For give start approximation $x_n$ we can calculate better approximation of the root:  
<center>$x_{n+1} = x_n - \frac{f(x)}{f'(x)} $</center>  
  
We can use this method to find root of $F'$, where is local optimum of $F$.  
  
For given approximation $x_n$ we can calculate better approximation of local optimum:  
<center>$x_{n+1} = x_n - \gamma [f''(x)]^{-1} f'(x) $</center>
<center>where $0<\gamma<1$ </center>
<center>$f'(x) = \nabla f(x) \in \mathbb {R} ^{d}$</center>
<center>$ f''(x)=\nabla ^{2}f(x)=H_{f}(x) \in \mathbb {R} ^{d\times d} $</center>  
$H_{f}$ is Hessian matrix and $\gamma$ is step size. We are trying $\gamma = \frac{1}{2^k}$ for $k=0...10$ until Wolfe conditions are satisfied.  
Wolfe conditions are:
\begin{aligned}{\textbf {i)}}&\quad f(\mathbf {x} _{n}+\gamma\mathbf {p} _{n})\leq f(\mathbf {x} _{n})+c_{1}\gamma\mathbf {p} _{n}^{\mathrm {T} }\nabla f(\mathbf {x} _{n}),\\[6pt]{\textbf {ii)}}&\quad {-\mathbf {p} }_{n}^{\mathrm {T} }\nabla f(\mathbf {x} _{n}+\gamma\mathbf {p} _{n})\leq -c_{2}\mathbf {p} _{n}^{\mathrm {T} }\nabla f(\mathbf {x} _{n}),\end{aligned}
<center>where ${p}_{n}=-\mathbf {H} ^{-1}\nabla f(\mathbf {x} _{n})$</center>  
<center>and c1,c2 are parameters, often equal $c_1=0.1$, $c_2=0.9$</center>

## SKlearn

In [53]:
LR_sklearn = LogisticRegression(solver='lbfgs')

In [54]:
LR_sklearn.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [55]:
preds_train_sklearn = LR.predict(X_train)

In [56]:
print_score(preds_train_sklearn,y_train,
                      'Train data, sklearn LR, C=1.0')

Train data, sklearn LR, C=1.0
Acc: 0.5403690888119954

Confusion matrix:
[[ 846  732]
 [ 862 1028]]

True negative (rating = 1): 846
True positive (rating = 10): 1028
False negative: 732
False positive: 862


(array([[ 846,  732],
        [ 862, 1028]], dtype=int64), 3468, 0.5403690888119954)

In [57]:
preds_test_sklearn = LR.predict(X_test)

In [58]:
print_score(preds_test_sklearn,y_test,
                      'Test data, sklearn LR, C=1.0')

Test data, sklearn LR, C=1.0
Acc: 0.5552995391705069

Confusion matrix:
[[189 155]
 [231 293]]

True negative (rating = 1): 189
True positive (rating = 10): 293
False negative: 155
False positive: 231


(array([[189, 155],
        [231, 293]], dtype=int64), 868, 0.5552995391705069)

## Testing different k

In [23]:
for K in range(1,11):
    X = np.array([[queue_imbalance_k(data,t,k) for k in range(1,K+1)] for t in T])
    X_train, X_test, y_train, y_test = train_test_split(
        X, target, test_size=0.2, random_state=42,shuffle=False)
    LR = LogisticRegression(solver = 'lbfgs')
    LR.fit(X_train,y_train)
    preds_train = LR.predict(X_train)
    print('k:', K)
    print(f'Train: {np.mean(preds_train == y_train)}')
    preds = LR.predict(X_test)
    print(f'Test: {np.mean(preds == y_test)}')
    print()

k: 1
Train: 0.5452710495963091
Test: 0.5403225806451613

k: 2
Train: 0.5403690888119954
Test: 0.5576036866359447

k: 3
Train: 0.5426758938869666
Test: 0.5564516129032258

k: 4
Train: 0.5475778546712803
Test: 0.5518433179723502

k: 5
Train: 0.5461361014994233
Test: 0.5495391705069125

k: 6
Train: 0.5452710495963091
Test: 0.5483870967741935

k: 7
Train: 0.5449826989619377
Test: 0.5483870967741935

k: 8
Train: 0.5446943483275664
Test: 0.5483870967741935

k: 9
Train: 0.5441176470588235
Test: 0.5460829493087558

k: 10
Train: 0.5446943483275664
Test: 0.5414746543778802



## Testing different C

In [24]:
for C in [0.01,0.05,0.1,0.2,0.25,0.3,0.4,0.5,0.75,0.85,0.9,1.0,1.25,1.5,2.0,5.0]:
    X = np.array([[queue_imbalance_k(data,t,k) for k in range(1,3)] for t in T])
    X_train, X_test, y_train, y_test = train_test_split(
        X, target, test_size=0.2, random_state=42,shuffle=False)
    LR = LogisticRegression(solver = 'lbfgs',C=C)
    LR.fit(X_train,y_train)
    preds_train = LR.predict(X_train)
    print('C:', C)
    print(f'Train: {np.mean(preds_train == y_train)}')
    preds = LR.predict(X_test)
    print(f'Test: {np.mean(preds == y_test)}')
    print()

C: 0.01
Train: 0.5432525951557093
Test: 0.5529953917050692

C: 0.05
Train: 0.5441176470588235
Test: 0.5576036866359447

C: 0.1
Train: 0.5426758938869666
Test: 0.5564516129032258

C: 0.2
Train: 0.5441176470588235
Test: 0.5552995391705069

C: 0.25
Train: 0.5441176470588235
Test: 0.5552995391705069

C: 0.3
Train: 0.5435409457900807
Test: 0.5576036866359447

C: 0.4
Train: 0.5426758938869666
Test: 0.5576036866359447

C: 0.5
Train: 0.5415224913494809
Test: 0.5576036866359447

C: 0.75
Train: 0.5403690888119954
Test: 0.5576036866359447

C: 0.85
Train: 0.5403690888119954
Test: 0.5576036866359447

C: 0.9
Train: 0.5403690888119954
Test: 0.5576036866359447

C: 1.0
Train: 0.5403690888119954
Test: 0.5576036866359447

C: 1.25
Train: 0.5403690888119954
Test: 0.5576036866359447

C: 1.5
Train: 0.5403690888119954
Test: 0.5576036866359447

C: 2.0
Train: 0.5403690888119954
Test: 0.5564516129032258

C: 5.0
Train: 0.5403690888119954
Test: 0.5552995391705069



# Receiver operating characteristic

### TODO

# Results

### TODO

# Possible improvements

### TODO